In [ ]:
# pip install beautifulsoup4 lxml


  Using cached beautifulsoup4-4.13.5-py3-none-any.whl.metadata (3.8 kB)
  Using cached lxml-6.0.1-cp313-cp313-win_amd64.whl.metadata (3.9 kB)
  Using cached soupsieve-2.8-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.5-py3-none-any.whl (105 kB)
Using cached lxml-6.0.1-cp313-cp313-win_amd64.whl (4.0 MB)
Using cached soupsieve-2.8-py3-none-any.whl (36 kB)

   ---------------------------------------- 0/3 [soupsieve]
   ---------------------------------------- 0/3 [soupsieve]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   --

In [ ]:
# !pip install elasticsearch

In [1]:
from bs4 import BeautifulSoup
import html as htmllib
import re

def extract_plain_text(fields):
    """
    fields = article.get("fields", {}) from Guardian API result.
    Returns plain cleaned text or None.
    """
    if not fields:
        return None


    # fallback to HTML body
    body_html = fields.get("body")
    if not body_html:
        return None

    soup = BeautifulSoup(body_html, "lxml")

    # remove unwanted tags (figures, scripts, asides, ads, nav, footer)
    for tag in soup(["script", "style", "aside", "figure", "iframe", "header", "footer", "nav"]):
        tag.decompose()


    # get text with paragraph spacing
    text = soup.get_text(separator=" ")
    text = htmllib.unescape(text)

    # collapse multiple newlines and whitespace
    text = re.sub(
    r"(?im)^\s*(Updated\s+at\s+)?\d{1,2}\.\d{2}\s*(am|pm)\s+BST\s*$", 
    "", 
    text)
    text = re.sub(r'\n\s*\n+', '\n\n', text)   # keep paragraph breaks
    lines = [ln.strip() for ln in text.splitlines()]
    clean = "\n".join([ln for ln in lines if ln])

    # optional: trim extremely long articles (e.g., keep first N chars)
    # clean = clean[:20000]

    return clean


In [2]:
import requests
import csv
from time import sleep

API_KEY = "022783ad-6e46-4ba2-af9a-3a7464467d02"
BASE_URL = "https://content.guardianapis.com/search"
PAGE_SIZE = 200  # max 200

def crawl_guardian(pages=1, output_file="guardian_articles.csv"):
    all_articles = []

    # open CSV for writing
    with open(output_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL)  # quote all fields
        writer.writerow(["id","title","url","section","published_at","trail_text","body"])

        for page in range(1, pages + 1):
            params = {
                "api-key": API_KEY,
                "page": page,
                "page-size": PAGE_SIZE,
                "show-fields": "trailText,body",
                "order-by": "newest"
            }
            resp = requests.get(BASE_URL, params=params)
            data = resp.json()

            results = data.get("response", {}).get("results", [])
            if not results:
                break

            for article in results:
                fields = article.get("fields", {})
                body = extract_plain_text(fields)
                row = [
                    article.get("id", ""),
                    article.get("webTitle", ""),
                    article.get("webUrl", ""),
                    article.get("sectionName", ""),
                    article.get("webPublicationDate", ""),
                    fields.get("trailText", ""),
                    body or ""
                ]
                writer.writerow(row)
                all_articles.append(row)

            print(f"Page {page} done, got {len(results)} articles.")
            sleep(1)  # polite delay

    print(f"Saved {len(all_articles)} articles to {output_file}")
    return all_articles


if __name__ == "__main__":
    all_articles = crawl_guardian(pages=30)  # adjust number of pages


Page 1 done, got 200 articles.
Page 2 done, got 200 articles.
Page 3 done, got 200 articles.
Page 4 done, got 200 articles.
Page 5 done, got 200 articles.
Page 6 done, got 200 articles.
Page 7 done, got 200 articles.
Page 8 done, got 200 articles.
Page 9 done, got 200 articles.
Page 10 done, got 200 articles.
Page 11 done, got 200 articles.
Page 12 done, got 200 articles.
Page 13 done, got 200 articles.
Page 14 done, got 200 articles.
Page 15 done, got 200 articles.
Page 16 done, got 200 articles.
Page 17 done, got 200 articles.
Page 18 done, got 200 articles.
Page 19 done, got 200 articles.
Page 20 done, got 200 articles.
Page 21 done, got 200 articles.
Page 22 done, got 200 articles.
Page 23 done, got 200 articles.
Page 24 done, got 200 articles.
Page 25 done, got 200 articles.
Page 26 done, got 200 articles.
Page 27 done, got 200 articles.
Page 28 done, got 200 articles.
Page 29 done, got 200 articles.
Page 30 done, got 200 articles.
Saved 6000 articles to guardian_articles.csv


In [ ]:
import pickle
with open("D:/DH/Senior/Paperboy/pickled_data/raw_crawl.pkl", "wb") as f:
    pickle.dump(all_articles, f)  
print(f"Pickled {len(all_articles)} stories.")

Pickled 6000 stories.


In [ ]:
import sys
sys.path.append("D:/DH/Senior")  # adjust to the actual parent folder
from Paperboy.api.BGEM3 import embed_bgem3

count = 0
for doc in all_articles:
    embedded = embed_bgem3(str(doc[1]) + "\n\n" + str(doc[4]))
    data = embedded.get("data", None)
    doc.append(data[0])
    print(f"Embedded {count} raw articles.")
    count += 1

Embedded 0 raw articles.
Embedded 1 raw articles.
Embedded 2 raw articles.
Embedded 3 raw articles.
Embedded 4 raw articles.
Embedded 5 raw articles.
Embedded 6 raw articles.
Embedded 7 raw articles.
Embedded 8 raw articles.
Embedded 9 raw articles.
Embedded 10 raw articles.
Embedded 11 raw articles.
Embedded 12 raw articles.
Embedded 13 raw articles.
Embedded 14 raw articles.
Embedded 15 raw articles.
Embedded 16 raw articles.
Embedded 17 raw articles.
Embedded 18 raw articles.
Embedded 19 raw articles.
Embedded 20 raw articles.
Embedded 21 raw articles.
Embedded 22 raw articles.
Embedded 23 raw articles.
Embedded 24 raw articles.
Embedded 25 raw articles.
Embedded 26 raw articles.
Embedded 27 raw articles.
Embedded 28 raw articles.
Embedded 29 raw articles.
Embedded 30 raw articles.
Embedded 31 raw articles.
Embedded 32 raw articles.
Embedded 33 raw articles.
Embedded 34 raw articles.
Embedded 35 raw articles.
Embedded 36 raw articles.
Embedded 37 raw articles.
Embedded 38 raw articl

KeyboardInterrupt: 

In [25]:
print(all_articles[425])

['sport/2025/sep/19/rugby-union-england-world-cup-france-semi-final', 'Pressure is a privilege for England who must deliver when it matters against France | Ugo Monye', 'https://www.theguardian.com/sport/2025/sep/19/rugby-union-england-world-cup-france-semi-final', 'Sport', '2025-09-19T15:00:49Z', 'Everything appears to be in place for the tournament hosts who will be judged on what happens in the next eight days', ['Without disrespecting any of the opposition the Red Roses have faced, England’s World Cup starts now. Since the draw John Mitchell and his players would have had a pretty good idea they would be locking horns with France for a place in the final. We’re into the sharp end of the tournament and, such are the resources available to England, it would be crazy if they hadn’t plotted for a semi-final against  Les Bleues  in advance.   There is plenty of expectation on the Red Roses but pressure is a privilege and they have dealt with everything that has come their way so far. Th

In [29]:
import psycopg2

def get_connection():
    return psycopg2.connect(
        host="localhost",
        port=5434,
        dbname="paperboy",
        user="minhhieu",
        password="minhhieu888"
    )

# UPSERT function
def upsert_articles(all_articles):
    sql = """
    INSERT INTO articles (id, title, url, section, published_at, trail_text, body, embedding)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (id) DO NOTHING
    """

    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.executemany(sql, all_articles)  # batch insert
        conn.commit()
    print(f"Upserted {len(all_articles)} articles.")


In [31]:
upsert_articles(all_articles)

Upserted 3000 articles.


In [ ]:
import pandas as pd

df = pd.read_csv("guardian_articles.csv")

# Show first 5 rows
for row in all_articles:
    print(f"Article {i+1}")
    print("Title:", row[1])
    print("Section:", row[2])
    print("Published at:", row[3])
    print("Trail text:", row[4])
    print("Body preview:", row[7], "...\n")


In [193]:
empty_text_count = 0
empty_full_text_count = 0

for article in all_articles:
    if article[7] == None:
        empty_text_count += 1
    if article[1] == "":
        empty_full_text_count += 1

total = len(df)
print(f"Total articles: {total}")
print(f"Articles with empty 'text': {empty_text_count} ({empty_text_count/total*100:.2f}%)")
print(f"Articles with empty 'full_text': {empty_full_text_count} ({empty_full_text_count/total*100:.2f}%)")

Total articles: 3000
Articles with empty 'text': 0 (0.00%)
Articles with empty 'full_text': 0 (0.00%)


In [1]:
import pickle

with open("pickled_data/articles.pkl", "wb") as f:
    pickle.dump(all_articles, f)

FileNotFoundError: [Errno 2] No such file or directory: 'pickled_data/articles.pkl'

In [3]:
from pathlib import Path
ROOT = Path("d:/DH/Senior/Paperboy/src")  # your project root
file_path = ROOT / "pickled_data" / "articles.pkl"

with open(file_path, "rb") as f:
    all_articles = pickle.load(f)

print("Loaded:", len(all_articles))

Loaded: 3000


In [12]:
print(all_articles[666])

['books/2025/sep/19/fierceland-by-omar-musa-review-poet-and-rappers-second-novel-pulses-with-life', 'Fierceland by Omar Musa review – poet and rapper’s second novel pulses with life', 'https://www.theguardian.com/books/2025/sep/19/fierceland-by-omar-musa-review-poet-and-rappers-second-novel-pulses-with-life', 'Books', '2025-09-18T15:00:21Z', 'Following two siblings who must grapple with their father’s legacy after his death, Fierceland is at its best when Musa plays with language', ['It’s 1998, and Roz and her brother Harun are waiting for the release of The Legend of Zelda: Ocarina of Time. They’ll be the first kids in Kota Kinabalu, the capital of Sabah in Malaysian Borneo, to play it, and they’re certain it will be the best video game of all time. The game turns on and Roz and Harun are “lost in a magical forest … [that] hums with danger and enchantment”.   Fierceland, the second novel from Australian-Malaysian poet, rapper, writer and artist Omar Musa, opens and closes in the fores

In [14]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

print("Ping:", es.ping())
print("Info:", es.info())


Ping: True
Info: {'name': 'b815d3b3561e', 'cluster_name': 'docker-cluster', 'cluster_uuid': '-BGY_U3LQTC48cNgRTgA6A', 'version': {'number': '8.15.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179', 'build_date': '2024-08-05T10:05:34.233336849Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [20]:
from elasticsearch import Elasticsearch

# connect to your cluster
es = Elasticsearch("http://localhost:9200")

INDEX_NAME = "paperboy"

# 1. Create index (if not exists)
def create_index_if_not_exists(dim=1024):
    if not es.indices.exists(index=INDEX_NAME):
        es.indices.create(
            index=INDEX_NAME,
            body={
                "mappings": {
                    "properties": {
                        "id": {"type": "keyword"},
                        "title": {"type": "text"},
                        "url": {"type": "keyword"},
                        "section": {"type": "keyword"},
                        "published_at": {"type": "date"},
                        "trail_text": {"type": "text"},
                        "body": {"type": "text"},
                        # BM25 works automatically on text fields
                        "embedding": {"type": "dense_vector", "dims": dim}
                    }
                }
            }
        )
        print(f"Index {INDEX_NAME} created")
    else:
        print(f"Index {INDEX_NAME} already exists")

# 2. Upsert document
def upsert_article(row: dict):
    doc_id = row["id"]  # use Guardian's id as unique key
    es.update(
        index=INDEX_NAME,
        id=doc_id,
        body={
            "doc": row,
            "doc_as_upsert": True
        }
    )
    print(f"Upserted article {doc_id}")


In [21]:
def row_to_doc(row):
    return {
        "id": row[0],
        "title": row[1],
        "url": row[2],
        "section": row[3],
        "published_at": row[4],
        "trail_text": row[5],
        "body": row[6],
        "embedding": row[7],  # already a list of floats
    }


In [22]:
for row in all_articles:
    upsert_article(row_to_doc(row))

Upserted article politics/live/2025/sep/22/reform-labour-keir-starmer-nigel-farage-immigration-latest-uk-politics-updates
Upserted article world/2025/sep/19/what-uk-recognition-palestine-state-will-mean
Upserted article film/2025/sep/22/a-night-like-this-review-interesting-nocturnal-connection-rendered-flat
Upserted article business/live/2025/sep/22/rachel-reeves-gatwick-second-runway-growth-jobs-environmental-disaster-business-live-news
Upserted article us-news/live/2025/sep/22/donald-trump-charlie-kirk-autism-us-politics-live-news-updates
Upserted article commentisfree/2025/sep/22/america-democray-leadership-robert-reich
Upserted article us-news/2025/sep/22/robert-redford-sid-wells-death-murder-search
Upserted article lifeandstyle/2025/sep/22/we-all-want-big-personal-change-at-the-push-of-a-button-but-here-is-how-to-actually-achieve-it
Upserted article environment/2025/sep/22/canada-towns-slug-it-out-over-whose-tide-is-highest-burntcoat-head-park-bay-of-fundy-nunavik-tasiujaq-leaf-ba

In [27]:
res = es.search(
    index=INDEX_NAME,
    query={"multi_match": {"query": "climate change", "fields": ["title", "body"]}}
)
print(res.get('hits').get('hits')[0]["_score"])

7.8822002
